### PREPROCESAMIENTO

In [5]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [6]:
#Cargar el csv
df = pd.read_csv("../bin/data_preprocess.csv")

In [7]:
df

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,...,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
0,Madrid,58000.0,RUTA 66,C,12.0,19990.0,37250.0,4633.0,1811.0,1429.0,...,320.0,215.0,8.7,11,2018,118.0,4.0,Turbo de geometría variable,BMW,SERIE 3
1,Madrid,34680.0,RUTA 66,ECO,12.0,12990.0,19300.0,3571.0,1627.0,1488.0,...,92.0,167.0,13.8,12,2021,NaN,3.0,Turbo,FIAT,500
2,Madrid,66933.0,AUTOMOTOR DURSAN,C,12.0,17390.0,26650.0,4377.0,1806.0,1590.0,...,260.0,182.0,11.9,4,2017,103.0,4.0,Turbo,NISSAN,QASHQAI
3,Sevilla,159000.0,Mules Car,C,12.0,8000.0,22425.0,4702.0,1809.0,1499.0,...,300.0,195.0,11.4,1,2017,99.0,4.0,Turbo,OPEL,ASTRA
4,Madrid,62695.0,CLICARS MADRID,0 EMISIONES,12.0,22990.0,44700.0,4545.0,1805.0,1685.0,...,193.0,162.0,10.9,7,2022,NaN,4.0,Turbo,MITSUBISHI,ECLIPSE CROSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34057,Alicante,28000.0,BMW HISPAMOVIL,ECO,24.0,79500.0,82000.0,4713.0,1897.0,1676.0,...,700.0,250.0,4.9,12,2023,NaN,6.0,Turbo de geometría variable,BMW,X3
34058,Murcia,12280.0,AUDI HUERTAS MOTOR MURCIA,C,12.0,24000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,2023,NaN,NaN,NaN,AUDI,A1
34059,Madrid,18744.0,STAR MADRID,0 EMISIONES,36.0,38990.0,57005.0,4684.0,1834.0,1667.0,...,375.0,160.0,9.2,8,2023,NaN,NaN,NaN,MERCEDES-BENZ,EQB
34060,Madrid,101353.0,CLICARS MADRID,C,12.0,13490.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,2018,NaN,NaN,NaN,RENAULT,SCENIC


In [8]:
#Calcular en porcentaje de nulos
porcentaje_nan = df.isna().mean() * 100
porcentaje_nan

id_provincia                0.020551
kilometraje                 0.184957
id_concesionario            0.000000
id_distintivo_ambiental     2.427926
garantia                    2.060948
precio_contado              6.262110
precio_nuevo               28.594915
largo                      29.549058
ancho                      29.505020
alto                       29.572544
capacidad_maletero         30.001174
num_plazas                 29.493277
batalla                    29.546122
peso                       24.194117
num_puertas                 0.989372
consumo_medio              48.951911
consumo_carretera          53.922259
consumo_urbano             53.913452
deposito                   35.535201
combustible                 1.006987
cilindrada                 34.225823
tipo_cambio                 0.000000
id_traccion                29.493277
num_marchas                26.243321
potencia_kw                29.408138
potencia_cv                 0.974693
par                        29.930715
v

In [9]:
#Funcion Imputacion de datos

def imputar_datos(df):

    #Separamos columnas numericas de categoricas
    columnas_numericas = df.select_dtypes(include=['float', 'int']).columns
    columnas_categoricas = df.select_dtypes(include=['object']).columns
    
    #Inicializar Knn para las columnas numericas
    knn_imputer = KNNImputer(n_neighbors=3)
    
    #Aplicar knn imputer a las columnas numericas
    df_numericas = df[columnas_numericas]
    df_imputado_numericas = knn_imputer.fit_transform(df_numericas)
    
    #Convertir los datos imputados a df
    df_imputado_numericas = pd.DataFrame(df_imputado_numericas, columns=columnas_numericas)
    
    #Inicializar SimpleImputer para las columnas categoricas
    simple_imputer = SimpleImputer(strategy='most_frequent')
    
    #Aplicar SimpleImputer para las columnas categoricas
    df_categoricas = df[columnas_categoricas]
    df_imputado_categoricas = simple_imputer.fit_transform(df_categoricas)
    
    #Convertir las columnas categoricas imputadas a df
    df_imputado_categoricas = pd.DataFrame(df_imputado_categoricas,columns=columnas_categoricas)
    
    #Combinamos los datos imputados
    df_imputado = pd.concat([df_imputado_numericas, df_imputado_categoricas], axis =1)
    
    #verificamos que no hay valores nulos
    print(df_imputado.isnull().sum())
    
    return df_imputado

In [ ]:
#Imputacion de datos
df_imputado = imputar_datos(df)

df_imputado
#Guardamos el df si es necesario
#df_imputado.to_csv('Coches-segunda-mano/bin/df_imputado.csv', index=False)